<a href="https://colab.research.google.com/github/claudialeguiza/AA1-TUIA-Kidonakis-Leguiza/blob/navegador/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.metrics import Precision
from keras.utils import to_categorical
import joblib
import warnings
warnings.simplefilter('ignore')


In [46]:
def mostrar_ciudades():
    print("Las ciudades sobre las cuales puede hacer consulta son las siguientes:","\n")
    print("1. Sidney")
    print("2. SidneyAirport")
    print("3. Melbourne")
    print("4. MelbournAirport")
    print("5. Canberra")
    print("6. Adelaide")
    print("7. MountGambier")
    print("8. Cobar")
    print("9. Dartmoor")
    print("10. Salir")
    return

In [47]:
def validar_ciudad(opcion):

    if opcion == "1":
        location = "Sidney"
    elif opcion == "2":
        location = "SidneyAirport"
    elif opcion == "3":
        location = "Melbourne"
    elif opcion == "4":
        location = "MelbourneAirport"
    elif opcion == "5":
       location = "Canberra"
    elif opcion == "6":
       location = "Adelaida"
    elif opcion == "7":
       location = "MontGambier"
    elif opcion == "8":
       location = "Cobar"
    elif opcion == "9":
       location = "Dartmoor"
    elif opcion == "10":
        print("Gracias por usar el predictor")
        return

    return location

In [48]:
def validar_temp(temp):
	try:
		decimal = float(temp)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= -10 and decimal <= 50)

In [49]:
def validar_presion(pres):
	try:
		decimal = float(pres)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 950 and decimal <= 1041)

In [50]:
def validar_nubosidad(nub):
	try:
		decimal = float(nub)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 10)

In [51]:
def validar_humedad(humed):
	try:
		decimal = float(humed)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 100)

In [52]:
def mostrar_opciones():
    print("Opciones direccion del viento permitidas:","\n")
    print("N - Norte   NE - Noreste  NW - Noroeste  NNE - Norte Noreste", "\n" )
    print("S - Sur     SE - Sureste  SW - Suroeste  SSE - Sur Sureste    SSW - Sur Suroeste" , "\n")
    print("E - Este    ESE - Este Sureste           ENE - Este Noreste", "\n" )
    print("W - Oeste   WSW - Oeste Suroeste         WNW - Oeste Noroeste", "\n" )

    return


In [53]:
def obtener_datos_clima(dato):

    lista_direcciones = ["N", "NE", "NW", "NNE", "S", "SE", "SW", "SSE", "E",\
                           "ESE", "ENE", "W", "WSW", "WNW", "SSW", ""]

    if dato == 'MinTemp':
       valor = input("Temperatura minima(grados C):")
       while (validar_temp(valor) == False):
           print("Temperatura no valida")
           valor = input("Temperatura minima (grados C):")
       return float(valor)

    if dato == 'MaxTemp':
       valor = input("Temperatura maxima(grados C):")
       while (validar_temp(valor) == False):
           print("Temperatura no valida")
           valor = input("Temperatura maxima (grados C):")
       return float(valor)

    if dato == 'Rainfall':
       valor = input("Lluvia caida (mm) - Ingrese 0 si no llovio:")
       while (valor < ' 0' and valor > '300') :
           print("Valor no valido")
           valor = input("Milimetros de lluvia caida(Ingrese 0 si no llovio):")
       return float(valor)

    if dato == 'Evaporation':
       valor = input("Evaporacion (mm):")
       while (valor < '0' and valor > '100'):
            print("Valor no valido")
            valor = input("Evaporacion (mm):")
       return float(valor)

    if dato == 'Sunshine':
       valor = input("Horas de Sol:")
       while  (valor < '0' and valor > '11'):
           print("Valor no valido")
           valor = input("Horas de Sol:")
       return float(valor)

    if dato == 'WindGustDir':
       valor = input("Direccion rafagas de viento fuerte (ultimas 24 hs):")
       while valor not in lista_direcciones:
             print("Direccion no valida")
             valor = input("Direccion rafagas de viento fuerte (ultimas 24 hs):")
       return valor

    if dato == 'WindGustSpeed':
       valor = input("Velocidad de las rafagas de viento fuerte(ultimas 24 hs - km/h):")
       while (valor < '0' and valor > '400'):
              print("Valor no valido")
              valor = input("Velocidad de las rafagas de viento fuerte(ultimas 24 hs - km/h):")
       return float(valor)

    if dato == 'WindDir9am':
       valor = input("Direccion del viento 9AM: ")
       while valor not in lista_direcciones:
              print("Direccion no valda")
              valor = input("Direccion del viento 9AM: ")
       return valor

    if dato == 'WindDir3pm':
       valor = input("Direccion del viento 3PM: ")
       while valor not in lista_direcciones:
             print("direccion no valida")
             valor = input("Direccion del viento 3PM: ")
       return valor

    if dato == 'WindSpeed9am':
       valor = input("Velocidad del viento 9AM (km/h): ")
       while (valor < '0' and valor > '400') :
             print("Valor no valido")
             valor = input("Velocidad del viento 9AM(km/h):")
       return float(valor)

    if dato == 'WindSpeed3pm':
       valor = input("Velocidad del viento 3PM (km/h):")
       while (valor < '0' and valor > '400'):
             print("Valor no valido")
             valor = input("Velocidad del viento 3PM(km/h):")
       return float(valor)

    if dato =='Humidity9am':
       valor = input('Humedad 9AM: ')
       while validar_humedad(valor) == False:
             print('Humedad no valida')
             valor = input('Humedad 9AM:')
       return float(valor)

    if dato == 'Humidity3pm':
       valor = input("Humedad 3PM:")
       while (validar_humedad(valor) == False):
           print('Humedad no valida')
           valor = input('Humedad 3PM: ')
       return float(valor)

    if dato == 'Pressure9am':
       valor = input("Presion 9AM:")
       while (validar_presion(valor) == False):
             print("Presion no valida")
             valor = input("presion 9 AM:")
       return float(valor)

    if dato == 'Pressure3pm':
       valor = input("Presion 3PM: ")
       while (validar_presion(valor) == False):
             print("Presion no valida")
             valor = input("Presion 3PM:")
       return float(valor)

    if dato == 'Cloud9am':
       valor = input("Nubosidad 9AM:")
       while (validar_nubosidad(valor) == False):
             print("Nubosidad no valida")
             valor = input("Nubosidad 9AM:")
       return float(valor)

    if dato == 'Cloud3pm':
       valor = input("Nubosidad 3PM:")
       while (validar_nubosidad(valor) == False):
             print("Nubosidad no valida")
             validar_otras = input("Nubosidad 3PM:")
       return float(valor)

    if dato =='Temp9am':
       valor = input("Temperatura 9AM:")
       while (validar_temp(valor) == False):
             print("Temperatura no valida")
             valor = input("Temperatura 9AM: ")
       return float(valor)

    if dato == 'Temp3pm':
       valor = input("Temperatura 3PM: ")
       while (validar_temp(valor) == False):
             print("Temperatura no valida")
             valor = input("Temperatura 3PM: ")
       return float(valor)

    if dato == 'RainToday':
       valor = input("Llueve hoy?:(Yes/No)")
       while (valor != 'Yes' and valor != 'No'):
           print("Respuesta no valida")
           valor = input("Llueve hoy?:(Yes/No)")
       return valor



In [54]:
def validar_opcion(valor):
	try:
		decimal = float(valor)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return decimal >= 0 and decimal <= 10

In [55]:
def obtener_prediccion(df_filtrada, modelo):
    prediccion = modelo.predict(df_filtrada)
    return prediccion


In [56]:
def interactuar_con_usuario():
    print("¡Bienvenido al predictor del clima de Australia para mañana!", "\n")

    armar_prediccion = {}
    now = datetime.datetime.now()
    fecha = now.strftime("%Y-%m-%d")

    mostrar_ciudades()
    opcion = input("Seleccione una opcion: ")
    while validar_opcion(opcion) == False:
           print ("opcion no valida")
           opcion = input("Seleccione una opcion: ")
    if opcion == '10':
       print ("Gracias por usar el predictor")
       return opcion

    else:

      print("Ingrese los siguientes datos para predecir", "\n")
      mostrar_opciones()
      armar_prediccion = {'Date': (fecha),
                        'Location': validar_ciudad(opcion),
                        'MinTemp': obtener_datos_clima("MinTemp"),
                        'MaxTemp': obtener_datos_clima("MaxTemp"),
                        'Rainfall': obtener_datos_clima("Rainfall"),
                        'Evaporation': obtener_datos_clima("Evaporation"),
                        'Sunshine': obtener_datos_clima("Sunshine"),
                        'WindGustDir': obtener_datos_clima("WindGustDir"),
                        'WindGustSpeed': obtener_datos_clima("WindGustSpeed"),
                        'WindDir9am': obtener_datos_clima("WindDir9am"),
                        'WindDir3pm': obtener_datos_clima("WindDir3pm"),
                        'WindSpeed9am': obtener_datos_clima("WindSpeed9am"),
                        'WindSpeed3pm': obtener_datos_clima("WindSpeed3pm"),
                        'Humidity9am': obtener_datos_clima("Humidity9am"),
                        'Humidity3pm': obtener_datos_clima("Humidity3pm"),
                        'Pressure9am': obtener_datos_clima("Pressure9am"),
                        'Pressure3pm': obtener_datos_clima("Pressure3pm"),
                        'Cloud9am': obtener_datos_clima("Cloud9am"),
                        'Cloud3pm': obtener_datos_clima("Cloud3pm"),
                        'Temp9am': obtener_datos_clima("Temp9am"),
                        'Temp3pm': obtener_datos_clima("Temp3pm"),
                        'RainToday' : obtener_datos_clima("RainToday")
                          }


In [57]:
def cargar_modelo_regresion():
    # Cargamos el modelo
      modelo_regresion = load_model('/content/regression_model.h5')
      return modelo_regresion

In [58]:
def cargar_modelo_clasificacion():
      modelo_clasif = load_model('/content/classification_model_optimized.h5')
      return modelo_clasif

In [59]:
# Cargamos el dataset
datos =  pd.read_csv('/content/weatherAUS.csv', delimiter = ",")
df = datos[datos.Location\
        .isin(( 'Sydney','SydneyAirport','Melbourne', 'MelbourneAirport',\
                'Canberra','Adelaide', 'MountGambier','Cobar', 'Dartmoor' ))]
# Cargamos los pipelines
pipeline_regresion = joblib.load('/content/regresion_pipeline.joblib')
pipeline_clasificacion = joblib.load('/content/clasificacion_pipeline.joblib')


dic_prediccion = interactuar_con_usuario()
if dic_prediccion == '10':
   exit()
else:
   df_prediccion =pd.DataFrame([dic_prediccion], index = [0])

   #Aplicamos los pipelines
   prediccion_clasificacion = pipeline_clasificacion.predict(df_prediccion)
   print(" La prediccion es ", prediccion_clasificacion)
   prediccion_regresion = pipeline_regresion.predict(df_prediccion)
   print(" Caeran aprox : ", prediccion_regresion, 'mm')

¡Bienvenido al predictor del clima de Australia para mañana! 

Las ciudades sobre las cuales puede hacer consulta son las siguientes: 

1. Sidney
2. SidneyAirport
3. Melbourne
4. MelbournAirport
5. Canberra
6. Adelaide
7. MountGambier
8. Cobar
9. Dartmoor
10. Salir
Seleccione una opcion: 4
Ingrese los siguientes datos para predecir 

Opciones direccion del viento permitidas: 

N - Norte   NE - Noreste  NW - Noroeste  NNE - Norte Noreste 

S - Sur     SE - Sureste  SW - Suroeste  SSE - Sur Sureste    SSW - Sur Suroeste 

E - Este    ESE - Este Sureste           ENE - Este Noreste 

W - Oeste   WSW - Oeste Suroeste         WNW - Oeste Noroeste 

Temperatura minima(grados C):10
Temperatura maxima(grados C):34
Lluvia caida (mm) - Ingrese 0 si no llovio:0
Evaporacion (mm):6
Horas de Sol:8
Direccion rafagas de viento fuerte (ultimas 24 hs):S
Velocidad de las rafagas de viento fuerte(ultimas 24 hs - km/h):40
Direccion del viento 9AM: SE
Direccion del viento 3PM: S
Velocidad del viento 9AM (km

AttributeError: 'FunctionTransformer' object has no attribute 'predict'